In [1]:
import mongo
from pymatgen.io.ase import AseAtomsAdaptor
import pickle

#dataset of docs taken from jupyter-dev using:
#     from gaspy import gasdb, defaults
#     import warnings
#     warnings.filterwarnings('ignore')

#     filters = defaults.adsorption_filters('CO')
#     #filters['results.energy'] = {'$gt': -3.5, '$lt': 9.0}
#     #filters['processed_data.movement_data.max_adsorbate_movement']['$lt'] = 4.0 #specifically for OOH vs OH

#     # Establish the fingerprints that are needed for the preprocessing
#     fingerprints = {}
#     fingerprints['atoms']='$atoms'
#     fingerprints['results']='$results'
#     fingerprints['max_surface_movement']='$processed_data.movement_data.max_surface_movement'
#     fingerprints['adsorption_site'] = '$initial_configuration.atoms.atoms'
#     # Pull the documents and then modify them so that they'll work with the preprocessor
#     docs = gasdb.get_adsorption_docs(['CO'],extra_fingerprints=fingerprints, filters=filters)

CO_docs = pickle.load(open('/home/zulissi/CO_docs.pkl','rb'))
CO_docs = CO_docs[:200]

Transform block to turn a GASpy doc into the necessary structure for CGCNN

In [2]:
from cgcnn.data import StructureData
import torch
import numpy as np

from sklearn.base import TransformerMixin

class StructureDataTransformer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        return
    
    def transform(self,X):
        target_list = [doc['energy'] for doc in CO_docs]
        structure_list = [AseAtomsAdaptor.get_structure(mongo.make_atoms_from_doc(doc)) for doc in X]
        return StructureData(structure_list, target_list, *self.args, **self.kwargs)

    def fit(self,*_):
        return self

CGCNN model with skorch to make it sklearn compliant

In [3]:
from cgcnn.model import CrystalGraphConvNet
from torch import nn
from skorch import NeuralNetRegressor
from torch.utils.data import Dataset, DataLoader
from cgcnn.data import collate_pool

#This is a little weird, one of the parameters basically needs to match the results 
# that are coming from the transform function, should be fixable 
SDT = StructureDataTransformer(atom_init_loc='/home/zulissi/software/cgcnn_sklearn/atom_init.json')
structures, _ = SDT.transform(CO_docs)[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    iterator_train__num_workers=4,
    iterator_train__pin_memory=True,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__collate_fn = collate_pool,
    max_epochs=10,
    lr=0.1,
    device='cuda'
)

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(SDT, net)

#y, energies
target_list = [doc['energy'] for doc in CO_docs]
y = torch.Tensor(np.array(target_list).reshape((len(target_list),1)))

#This is hacky, I'm not sure if y is actually necessary, I think it's just buried
# in the CO_docs, but need to check
pipe.fit(CO_docs,y=y)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1      441.5199     1242.7625  40.3099
      2       69.0057       14.5083  29.6966
      3       10.7678        6.5655  29.8524
